In [2]:
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
import metos3d.petsc_mod as petsc
from os import path
import pandas as pd

In [3]:
data_path = r"D:\Bachelorarbeit Data\0000\save"
X = np.load(join(data_path,"c.npy" ))
data = np.load(join(data_path, "data.npy"))
mean_data = np.mean(data, axis=1)

## Function to evaluate the maximal data, does not 

In [4]:
def accumulate_data(data, index_set=None):
    if index_set is None:
        index_set = range(1, len(data[0]))

    acc = 0
    for row in data:
        for index in index_set:
            a_coef = abs(row[index])
            acc += a_coef * a_coef

    return acc

In [5]:
og = accumulate_data(X)

## Taking the maximal coefficient for each point
This does not work!!
Even when using every max index, this does not nearly come close enough to an approcimate quality

In [6]:

# index for the maximum in each row, ommiting the 0-th coefficient
maximums = np.argmax(X[:,1:], axis=1)
#increment each index
maximums = np.array(list(map(lambda x : x+1, maximums)))

# count unique indices and count how often each one comes up
idx, counts = np.unique(maximums, return_counts=True)
# sort the indices according to how often they are maximum
tuples = list(zip(idx, counts))
tuples.sort(key=lambda x : x[1], reverse=True)
# 
sort_idx = list(map(lambda x: x[0], tuples))


In [7]:
deltas = []
idx_num = []
used_coefs = []

for i in range(1,11):
    comp = accumulate_data(X, index_set=set(sort_idx[:i]))
    deltas.append(comp / og)
    idx_num.append(i)
    used_coefs.append(sort_idx[:i])
df = pd.DataFrame({"# of coef": idx_num,"used coefs":used_coefs, "delta": deltas})
df

,# of coef,used coefs,delta
0,1,[1],0.458948
1,2,"[1, 2]",0.488431
2,3,"[1, 2, 2879]",0.947380
3,4,"[1, 2, 2879, 3]",0.953241
4,5,"[1, 2, 2879, 3, 4]",0.955534
5,6,"[1, 2, 2879, 3, 4, 2878]",0.985017
6,7,"[1, 2, 2879, 3, 4, 2878, 2877]",0.990878
7,8,"[1, 2, 2879, 3, 4, 2878, 2877, 2875]",0.992034
8,9,"[1, 2, 2879, 3, 4, 2878, 2877, 2875, 2874]",0.992694
9,10,"[1, 2, 2879, 3, 4, 2878, 2877, 2875, 2874, 2876]",0.994987


## Get the best indices for this measure

get each value for $C_k$ for coefficients $c_k$ for point $p$ with 
$$ C_k = \sum_{p \in P} |c_k^p| ^2$$

In [8]:
C_k = [0] * len(X[0])
for row in X:
    for i in range(1,len(row)):
        abs_c = abs(row[i])
        C_k[i] += abs_c * abs_c

In [9]:
C_sorted = sorted(enumerate(C_k), reverse=True, key=lambda c: c[1])
C_idx_sorted = list(map(lambda c: c[0],C_sorted))


In [10]:
i = 10
indices = map(lambda x : x[0], C_sorted[:i])
deltas = map(lambda x: x[1] / og, C_sorted[:i])
pd.DataFrame({"index": indices, "\delta_i": deltas})

,index,\delta_i
0,1,0.458948
1,2879,0.458948
2,2,0.029483
3,2878,0.029483
4,3,0.005861
5,2877,0.005861
6,4,0.002293
7,2876,0.002293
8,5,0.001156
9,2875,0.001156


In [11]:
delta = []
index_set = []
number = []

for i in range(1,11):
    comp = accumulate_data(X, index_set=C_idx_sorted[:i])
    delta.append(comp / og)
    index_set.append(C_idx_sorted[:i])
    number.append(i)

pd.DataFrame({"# of index":number, "index set": index_set, "delta": delta})

,# of index,index set,delta
0,1,[1],0.458948
1,2,"[1, 2879]",0.917897
2,3,"[1, 2879, 2]",0.947380
3,4,"[1, 2879, 2, 2878]",0.976863
4,5,"[1, 2879, 2, 2878, 3]",0.982724
5,6,"[1, 2879, 2, 2878, 3, 2877]",0.988585
6,7,"[1, 2879, 2, 2878, 3, 2877, 4]",0.990878
7,8,"[1, 2879, 2, 2878, 3, 2877, 4, 2876]",0.993170
8,9,"[1, 2879, 2, 2878, 3, 2877, 4, 2876, 5]",0.994327
9,10,"[1, 2879, 2, 2878, 3, 2877, 4, 2876, 5, 2875]",0.995483


Tabelle machen , gerade Zahlen


## Compare compressed Data to original


In [12]:
def fake_compress (data, index_set):
    compressed_data = np.zeros(data.shape, dtype="complex128")
    for i in index_set:
        compressed_data[:,i] = data[:,i]

    return np.abs(np.real(np.fft.ifft(compressed_data)))

In [13]:
land_sea_mask = petsc.read_PETSc_matrix('metos3d\\landSeaMask.petsc')
n1, n2 = np.shape(land_sea_mask.T)
mean_values = np.mean(data, axis=1)

In [28]:
def get_min_max(data1, data2, data3):

    min1 = np.min(data1, axis=1) - mean_values
    min2 = np.min(data2, axis=1) - mean_values
    min3 = np.min(data3, axis=1) - mean_values

    max1 = np.max(data1, axis=1) - mean_values
    max2 = np.max(data2, axis=1) - mean_values
    max3 = np.max(data3, axis=1) - mean_values

    maxes = []
    mins = []
    
    min1, _ , _, _ = petsc.reshape_vector_to_3d(land_sea_mask,min1)
    min2, _ , _, _ = petsc.reshape_vector_to_3d(land_sea_mask,min2)
    min3, _ , _, _ = petsc.reshape_vector_to_3d(land_sea_mask,min3)

    max1, _ , _, _ = petsc.reshape_vector_to_3d(land_sea_mask,max1)
    max2, _ , _, _ = petsc.reshape_vector_to_3d(land_sea_mask,max2)
    max3, _ , _, _ = petsc.reshape_vector_to_3d(land_sea_mask,max3)

    for i in range(15):
        max_elem = max(
                np.nanmax(max1[:,:,i]),
                np.nanmax(max2[:,:,i]),
                np.nanmax(max3[:,:,i]),
            )
        maxes.append(max_elem)

        print( np.nanmin(min1[:,:,i]),
                np.nanmin(min2[:,:,i]),
                np.nanmin(min3[:,:,i]),
            )

        mins.append(
            min(
                np.nanmin(min1[:,:,i]),
                np.nanmin(min2[:,:,i]),
                np.nanmin(min3[:,:,i]),
            )
        )

    return mins, maxes

mins, maxes = get_min_max(data, fake_compress(X,index_set=([0] + C_idx_sorted[:2])), fake_compress(X, index_set=([0] + C_idx_sorted[:4])))

-0.3588024804138362 -0.3991645833989094 -0.3814207382905417
-0.5044725329575495 -0.5116384967585683 -0.5049238412129586
-0.2915858103122073 -0.26593679800309666 -0.27618884868966265
-0.15403837087554217 -0.14463551667988828 -0.17292532429021867
-0.07124092896484413 -0.06882142357173637 -0.08288249003143378
-0.0650681489946453 -0.06335249599789028 -0.06443772800018066
-0.0399849453051917 -0.03682965249011527 -0.03896430293148967
-0.0417718142670358 -0.04265078224989782 -0.04233989643692748
-0.025077195940862218 -0.025580952467473494 -0.025452904063444137
-0.009743323571322682 -0.008685929050003649 -0.009099856331739709
-0.015723550955122123 -0.013815432474988842 -0.014598690739356046
-0.02269708738677112 -0.02039484936791247 -0.021420935652249673
-0.008425337050183668 -0.0075327181347186745 -0.00788180846754205
-0.007163092190491938 -0.0070813518399450714 -0.006867365106664636
-0.0031223140201706023 -0.0029269151794681925 -0.003002959554859519


In [31]:
list(zip(mins,maxes))

[(-0.3991645833989094, 0.528397944443761),
 (-0.5116384967585683, 0.640772956323747),
 (-0.2915858103122073, 0.3210485840893387),
 (-0.17292532429021867, 0.1702051132414928),
 (-0.08288249003143378, 0.09205707551395892),
 (-0.0650681489946453, 0.06335249599788706),
 (-0.0399849453051917, 0.03816639522484344),
 (-0.04265078224989782, 0.04389701742953278),
 (-0.025580952467473494, 0.026395464274459712),
 (-0.009743323571322682, 0.009527571211308539),
 (-0.015723550955122123, 0.014990954103520027),
 (-0.02269708738677112, 0.021589829989564047),
 (-0.008425337050183668, 0.008691129114050344),
 (-0.007163092190491938, 0.007981563414362736),
 (-0.0031223140201706023, 0.003213656009288357)]

In [32]:
relative_levels = [
   list(np.linspace(lower, upper, 10)) for lower,upper in zip(mins, maxes) 
]

In [33]:
def plot_series(data, steps, save_path, land_sea_mask, layer, mean_values=None):
    stepsize = len(data[0]) // steps
    n1, n2 = np.shape(land_sea_mask.T)
    long, lat = np.meshgrid(np.linspace(-90, 90, n2), np.linspace(0, 360, n1))


    for step in range(steps):
        layer_data = data[:, step * stepsize]

        if not mean_values is None:
            layer_data = layer_data - mean_values

        fig, ax = plt.subplots(figsize=(15, 7))
        data_3d, _ , _, _ = petsc.reshape_vector_to_3d(land_sea_mask,layer_data)

        plot = ax.contourf(lat, long, data_3d[:, :, layer], levels=None if mean_values is None else relative_levels[layer])
        fig.colorbar(plot, ax=ax)
        ax.set_ylabel('latitude')
        ax.set_xlabel('longitude')

        fig.savefig(join(save_path, f"{step}.png"))
        plt.close(fig)

In [35]:
for layer in range(15):
        print(layer)
        plot_series(data, 48, r"D:\Bachelorarbeit Data\bilder\relative_values\reference\layer" + str(layer), land_sea_mask, layer, mean_values=mean_values)
        # plot_series(data, 48, r"D:\Bachelorarbeit Data\bilder\absolute values\reference\layer" + str(layer), land_sea_mask, layer)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [36]:
for i in [2,4]:
    index_set = [0] + C_idx_sorted[:i]
    for layer in range(15):
        print(i,layer)
        plot_series(fake_compress(X, index_set), 48, r"D:\Bachelorarbeit Data\bilder\relative_values\perfect_" + str(i) + r"\layer" + str(layer), land_sea_mask, layer, mean_values=mean_values)
        # plot_series(fake_compress(X, index_set), 48, r"D:\Bachelorarbeit Data\bilder\absolute values\perfect_" + str(i) + r"\layer" + str(layer), land_sea_mask, layer)

2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
